In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import sys

sys.path.append("..")
from model import Net

In [4]:
def get_model_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f'Total Parameters: {total_params}')
    return total_params

model = Net()
get_model_params(model)


Total Parameters: 12866


12866

In [5]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
         ConvBlock-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,168
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         Dropout2d-7           [-1, 16, 28, 28]               0
         ConvBlock-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 32, 28, 28]           4,640
      BatchNorm2d-10           [-1, 32, 28, 28]              64
        Dropout2d-11           [-1, 32, 28, 28]               0
        ConvBlock-12           [-1, 32, 28, 28]               0
        MaxPool2d-13           [-1, 32, 14, 14]               0
           Conv2d-14            [-1, 4,

In [6]:
torch.manual_seed(1)
batch_size = 128

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.RandomRotation(10),  # Rotate by up to 10 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Translate by up to 10% in each direction
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Apply perspective transform
])

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)


In [7]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        # Clamp outputs
        output = torch.clamp(output, min=-100, max=100)

        loss = F.nll_loss(output, target)

        if torch.isnan(loss):
            print(f"Loss is NaN at batch {batch_idx}")
            continue

        accuracy = (output.argmax(dim=1) == target).float().mean()
        loss.backward()

        # Calculate gradient norm before clipping
        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        
        # Log if gradient norm is large
        if total_norm > 4.0:  # Monitor when gradients are getting large
            print(f'Gradient norm: {total_norm:.2f}')

        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch}; loss={loss.item():.4f}; accuracy={accuracy.item():.3f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        pbar = tqdm(test_loader)
        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(device), target.to(device)
            output = model(data)

            # Clamp outputs
            output = torch.clamp(output, min=-100, max=100)

            test_loss += F.nll_loss(output, target, reduction='sum')  # sum up batch loss

            if torch.isnan(test_loss):
                print(f"Test loss is NaN at batch {batch_idx}")
                continue

            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss.item(), correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)
epochs = 20

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch=1; loss=2.2658; accuracy=0.172:  26%|██▌       | 246/938 [00:21<01:00, 11.38it/s]


KeyboardInterrupt: 